# سير عمل الإنسان في الحلقة باستخدام إطار عمل Microsoft Agent

## 🎯 أهداف التعلم

في هذا الدفتر، ستتعلم كيفية تنفيذ سير عمل **الإنسان في الحلقة** باستخدام `RequestInfoExecutor` في إطار عمل Microsoft Agent. يتيح هذا النمط القوي إيقاف سير عمل الذكاء الاصطناعي لجمع مدخلات بشرية، مما يجعل الوكلاء تفاعليين ويمنح البشر التحكم في القرارات المهمة.

## 🔄 ما هو الإنسان في الحلقة؟

**الإنسان في الحلقة (HITL)** هو نمط تصميم حيث يتوقف وكلاء الذكاء الاصطناعي عن التنفيذ لطلب مدخلات بشرية قبل المتابعة. هذا ضروري لـ:

- ✅ **القرارات الحاسمة** - الحصول على موافقة بشرية قبل اتخاذ إجراءات مهمة
- ✅ **المواقف الغامضة** - السماح للبشر بالتوضيح عندما يكون الذكاء الاصطناعي غير متأكد
- ✅ **تفضيلات المستخدم** - طلب من المستخدمين اختيار بين خيارات متعددة
- ✅ **الامتثال والسلامة** - ضمان إشراف بشري للعمليات المنظمة
- ✅ **تجارب تفاعلية** - بناء وكلاء محادثة يستجيبون لمدخلات المستخدم

## 🏗️ كيف يعمل في إطار عمل Microsoft Agent

يوفر الإطار ثلاثة مكونات رئيسية لـ HITL:

1. **`RequestInfoExecutor`** - منفذ خاص يوقف سير العمل ويصدر حدث `RequestInfoEvent`
2. **`RequestInfoMessage`** - فئة أساسية لحمولات الطلب الموجهة للبشر
3. **`RequestResponse`** - يربط ردود البشر بالطلبات الأصلية باستخدام `request_id`

**نمط سير العمل:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 مثالنا: حجز فندق مع تأكيد المستخدم

سنقوم بالبناء على سير العمل الشرطي بإضافة تأكيد بشري **قبل** اقتراح وجهات بديلة:

1. يطلب المستخدم وجهة (مثل "باريس")
2. يتحقق `availability_agent` من توفر الغرف
3. **إذا لم تكن هناك غرف** → يسأل `confirmation_agent` "هل ترغب في رؤية البدائل؟"
4. يتوقف سير العمل باستخدام `RequestInfoExecutor`
5. **يرد الإنسان** بـ "نعم" أو "لا" عبر إدخال في وحدة التحكم
6. يقوم `decision_manager` بتوجيه بناءً على الرد:
   - **نعم** → عرض وجهات بديلة
   - **لا** → إلغاء طلب الحجز
7. عرض النتيجة النهائية

هذا المثال يوضح كيفية منح المستخدمين التحكم في اقتراحات الوكيل!

---

لنبدأ! 🚀


## الخطوة 1: استيراد المكتبات المطلوبة

نقوم باستيراد مكونات إطار العمل القياسي للوكيل بالإضافة إلى **الفئات الخاصة بالتفاعل البشري**:
- `RequestInfoExecutor` - منفذ يوقف سير العمل للحصول على إدخال بشري
- `RequestInfoEvent` - حدث يتم إصداره عند طلب إدخال بشري
- `RequestInfoMessage` - الفئة الأساسية لحمولات الطلب الموجهة
- `RequestResponse` - يربط بين استجابات البشر والطلبات
- `WorkflowOutputEvent` - حدث للكشف عن مخرجات سير العمل


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## الخطوة 2: تعريف نماذج Pydantic للمخرجات المنظمة

تحدد هذه النماذج **المخطط** الذي ستعيده الوكلاء. نحافظ على جميع النماذج من سير العمل الشرطي ونضيف:

**جديد للتفاعل البشري:**
- `HumanFeedbackRequest` - فئة فرعية من `RequestInfoMessage` التي تحدد حمولة الطلب المرسلة للبشر
  - تحتوي على `prompt` (السؤال المطلوب طرحه) و`destination` (السياق حول المدينة غير المتوفرة)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## الخطوة 3: إنشاء أداة حجز الفنادق

نفس الأداة من سير العمل الشرطي - تتحقق من توفر الغرف في الوجهة.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## الخطوة 4: تعريف وظائف الشروط للتوجيه

نحتاج إلى **أربع وظائف شروط** لعملية العمل التي تتضمن الإنسان في الحلقة:

**من عملية العمل الشرطية:**
1. `has_availability_condition` - يوجه عندما تكون الفنادق متوفرة
2. `no_availability_condition` - يوجه عندما تكون الفنادق غير متوفرة

**جديد لعملية الإنسان في الحلقة:**
3. `user_wants_alternatives_condition` - يوجه عندما يقول المستخدم "نعم" للخيارات البديلة
4. `user_declines_alternatives_condition` - يوجه عندما يقول المستخدم "لا" للخيارات البديلة


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## الخطوة 5: إنشاء منفذ مدير القرارات

هذه هي **جوهر نمط الإنسان في الحلقة**! `DecisionManager` هو منفذ مخصص يقوم بـ:

1. **استقبال ملاحظات الإنسان** عبر كائنات `RequestResponse`
2. **معالجة قرار المستخدم** (نعم/لا)
3. **توجيه سير العمل** عن طريق إرسال رسائل إلى الوكلاء المناسبين

الميزات الرئيسية:
- يستخدم الزخرفة `@handler` لعرض الطرق كخطوات في سير العمل
- يستقبل `RequestResponse[HumanFeedbackRequest, str]` الذي يحتوي على الطلب الأصلي وإجابة المستخدم
- ينتج رسائل بسيطة "نعم" أو "لا" التي تُشغل وظائف الشرط الخاصة بنا


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## الخطوة 6: إنشاء منفذ عرض مخصص

نفس منفذ العرض من سير العمل الشرطي - يقدم النتائج النهائية كمخرجات سير العمل.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## الخطوة 7: تحميل متغيرات البيئة

قم بتكوين عميل LLM (نماذج GitHub، Azure OpenAI، أو OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## الخطوة 8: إنشاء وكلاء الذكاء الاصطناعي والمنفذين

نقوم بإنشاء **ستة مكونات للعملية**:

**الوكلاء (مغلفين داخل AgentExecutor):**
1. **availability_agent** - يتحقق من توفر الفنادق باستخدام الأداة
2. **confirmation_agent** - 🆕 يُجهز طلب تأكيد من الإنسان
3. **alternative_agent** - يقترح مدن بديلة (عندما يقول المستخدم نعم)
4. **booking_agent** - يشجع على الحجز (عندما تكون الغرف متوفرة)
5. **cancellation_agent** - 🆕 يتعامل مع رسالة الإلغاء (عندما يقول المستخدم لا)

**المنفذون الخاصون:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` الذي يوقف سير العمل للحصول على إدخال من الإنسان
7. **decision_manager** - 🆕 منفذ مخصص يقوم بالتوجيه بناءً على استجابة الإنسان (تم تعريفه بالفعل أعلاه)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## الخطوة 9: بناء سير العمل مع تدخل الإنسان

الآن نقوم ببناء مخطط سير العمل مع **التوجيه الشرطي** بما في ذلك مسار تدخل الإنسان:

**هيكل سير العمل:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**الحواف الرئيسية:**
- `availability_agent → confirmation_agent` (عندما لا توجد غرف)
- `confirmation_agent → prepare_human_request` (تغيير النوع)
- `prepare_human_request → request_info_executor` (توقف لانتظار الإنسان)
- `request_info_executor → decision_manager` (دائمًا - يوفر RequestResponse)
- `decision_manager → alternative_agent` (عندما يقول المستخدم "نعم")
- `decision_manager → cancellation_agent` (عندما يقول المستخدم "لا")
- `availability_agent → booking_agent` (عندما تكون الغرف متوفرة)
- جميع المسارات تنتهي عند `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## الخطوة 10: تشغيل حالة الاختبار 1 - مدينة بدون توفر (باريس مع تأكيد بشري)

هذا الاختبار يوضح **الدورة الكاملة للتفاعل البشري**:

1. طلب فندق في باريس
2. يقوم availability_agent بالتحقق → لا توجد غرف
3. يقوم confirmation_agent بإنشاء سؤال موجه للبشر
4. يقوم request_info_executor **بإيقاف سير العمل مؤقتًا** ويصدر `RequestInfoEvent`
5. **التطبيق يكتشف الحدث ويطلب من المستخدم عبر وحدة التحكم**
6. يقوم المستخدم بكتابة "نعم" أو "لا"
7. يرسل التطبيق الرد عبر `send_responses_streaming()`
8. يقوم decision_manager بتوجيه العملية بناءً على الرد
9. يتم عرض النتيجة النهائية

**النمط الرئيسي:**
- استخدم `workflow.run_stream()` للتكرار الأول
- استخدم `workflow.send_responses_streaming(pending_responses)` للتكرارات اللاحقة
- استمع إلى `RequestInfoEvent` لاكتشاف متى تكون هناك حاجة إلى إدخال بشري
- استمع إلى `WorkflowOutputEvent` لالتقاط النتائج النهائية


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## الخطوة 11: تشغيل حالة الاختبار 2 - مدينة مع توفر (ستوكهولم - لا حاجة لتدخل بشري)

يُظهر هذا الاختبار **المسار المباشر** عندما تكون الغرف متوفرة:

1. طلب فندق في ستوكهولم  
2. يتحقق availability_agent → الغرف متوفرة ✅  
3. يقترح booking_agent الحجز  
4. يعرض display_result التأكيد  
5. **لا حاجة لتدخل بشري!**

يتجاوز سير العمل المسار الذي يتطلب تدخل بشري تمامًا عندما تكون الغرف متوفرة.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## النقاط الرئيسية وأفضل الممارسات للتفاعل البشري في العمليات

### ✅ ما تعلمته:

#### 1. **نمط RequestInfoExecutor**
نمط التفاعل البشري في إطار عمل Microsoft Agent يعتمد على ثلاثة مكونات رئيسية:
- `RequestInfoExecutor` - يوقف سير العمل ويصدر أحداثًا
- `RequestInfoMessage` - الفئة الأساسية لحمولات الطلبات المخصصة (قم بتوريثها!)
- `RequestResponse` - يربط ردود البشر بالطلبات الأصلية

**فهم أساسي:**
- `RequestInfoExecutor` لا يجمع المدخلات بنفسه - فقط يوقف سير العمل
- يجب أن تستمع شفرة تطبيقك إلى `RequestInfoEvent` لجمع المدخلات
- يجب عليك استدعاء `send_responses_streaming()` مع قاموس يربط `request_id` بإجابة المستخدم

#### 2. **نمط التنفيذ المتدفق**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **الهيكلية القائمة على الأحداث**
استمع إلى أحداث محددة للتحكم في سير العمل:
- `RequestInfoEvent` - مطلوب إدخال بشري (سير العمل متوقف)
- `WorkflowOutputEvent` - النتيجة النهائية متاحة (سير العمل مكتمل)
- `WorkflowStatusEvent` - تغييرات الحالة (IN_PROGRESS، IDLE_WITH_PENDING_REQUESTS، إلخ)

#### 4. **منفذون مخصصون باستخدام @handler**
يوضح `DecisionManager` كيفية إنشاء منفذين يقومون بـ:
- استخدام الزخرفة `@handler` لجعل الطرق خطوات في سير العمل
- استقبال رسائل مخصصة (مثل `RequestResponse[HumanFeedbackRequest, str]`)
- توجيه سير العمل عن طريق إرسال رسائل إلى منفذين آخرين
- الوصول إلى السياق عبر `WorkflowContext`

#### 5. **التوجيه الشرطي بقرارات بشرية**
يمكنك إنشاء وظائف شرطية لتقييم ردود البشر:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 تطبيقات واقعية:

1. **سير العمل للموافقة**
   - الحصول على موافقة المدير قبل معالجة تقارير النفقات
   - طلب مراجعة بشرية قبل إرسال رسائل البريد الإلكتروني التلقائية
   - تأكيد المعاملات ذات القيمة العالية قبل التنفيذ

2. **مراجعة المحتوى**
   - الإشارة إلى المحتوى المشكوك فيه للمراجعة البشرية
   - طلب من المشرفين اتخاذ القرار النهائي في الحالات الحدودية
   - التصعيد للبشر عندما تكون ثقة الذكاء الاصطناعي منخفضة

3. **خدمة العملاء**
   - السماح للذكاء الاصطناعي بمعالجة الأسئلة الروتينية تلقائيًا
   - تصعيد القضايا المعقدة إلى وكلاء بشريين
   - سؤال العميل إذا كان يريد التحدث إلى إنسان

4. **معالجة البيانات**
   - طلب من البشر حل إدخالات البيانات الغامضة
   - تأكيد تفسيرات الذكاء الاصطناعي للوثائق غير الواضحة
   - السماح للمستخدمين بالاختيار بين تفسيرات متعددة صحيحة

5. **أنظمة السلامة الحرجة**
   - طلب تأكيد بشري قبل اتخاذ إجراءات لا رجعة فيها
   - الحصول على الموافقة قبل الوصول إلى البيانات الحساسة
   - تأكيد القرارات في الصناعات المنظمة (الرعاية الصحية، المالية)

6. **الوكلاء التفاعليون**
   - بناء روبوتات محادثة تسأل أسئلة متابعة
   - إنشاء معالجات ترشد المستخدمين خلال العمليات المعقدة
   - تصميم وكلاء يتعاونون مع البشر خطوة بخطوة

### 🔄 مقارنة: مع أو بدون التفاعل البشري

| الميزة | سير العمل الشرطي | سير العمل مع التفاعل البشري |
|---------|---------------------|---------------------------|
| **التنفيذ** | `workflow.run()` واحد | حلقة مع `run_stream()` + `send_responses_streaming()` |
| **مدخلات المستخدم** | لا شيء (تلقائي بالكامل) | مطالبات تفاعلية عبر `input()` أو واجهة المستخدم |
| **المكونات** | وكلاء + منفذون | + RequestInfoExecutor + DecisionManager |
| **الأحداث** | فقط AgentExecutorResponse | RequestInfoEvent، WorkflowOutputEvent، إلخ |
| **التوقف** | لا توقف | سير العمل يتوقف عند RequestInfoExecutor |
| **التحكم البشري** | لا يوجد تحكم بشري | البشر يتخذون قرارات رئيسية |
| **حالة الاستخدام** | التشغيل الآلي | التعاون والإشراف |

### 🚀 أنماط متقدمة:

#### نقاط قرار بشرية متعددة
يمكنك أن يكون لديك عدة عقد `RequestInfoExecutor` في نفس سير العمل:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### معالجة المهلة الزمنية
قم بتنفيذ مهلات لردود البشر:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### تكامل واجهة المستخدم الغنية
بدلاً من `input()`، قم بالتكامل مع واجهة الويب، Slack، Teams، إلخ:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### التفاعل البشري الشرطي
اطلب إدخالًا بشريًا فقط في مواقف محددة:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ أفضل الممارسات:

1. **قم دائمًا بتوريث RequestInfoMessage**
   - يوفر أمان النوع والتحقق
   - يتيح سياقًا غنيًا لتقديم واجهة المستخدم
   - يوضح نية كل نوع طلب

2. **استخدم مطالبات وصفية**
   - قم بتضمين السياق حول ما تسأل عنه
   - اشرح عواقب كل اختيار
   - اجعل الأسئلة بسيطة وواضحة

3. **تعامل مع المدخلات غير المتوقعة**
   - تحقق من صحة ردود المستخدم
   - قدم افتراضات للمدخلات غير الصالحة
   - قدم رسائل خطأ واضحة

4. **تتبع معرفات الطلبات**
   - استخدم العلاقة بين request_id والردود
   - لا تحاول إدارة الحالة يدويًا

5. **صمم لعدم الحظر**
   - لا تحظر الخيوط أثناء انتظار المدخلات
   - استخدم أنماط غير متزامنة طوال الوقت
   - دعم مثيلات سير العمل المتزامنة

### 📚 مفاهيم ذات صلة:

- **وسيط الوكيل** - اعتراض مكالمات الوكيل (دفتر الملاحظات السابق)
- **إدارة حالة سير العمل** - الحفاظ على حالة سير العمل بين التشغيلات
- **التعاون بين الوكلاء المتعددين** - الجمع بين التفاعل البشري وفِرق الوكلاء
- **الهياكل القائمة على الأحداث** - بناء أنظمة تفاعلية مع الأحداث

---

### 🎓 تهانينا!

لقد أتقنت سير العمل التفاعلي البشري مع إطار عمل Microsoft Agent! الآن تعرف كيفية:
- ✅ إيقاف سير العمل لجمع المدخلات البشرية
- ✅ استخدام RequestInfoExecutor و RequestInfoMessage
- ✅ التعامل مع التنفيذ المتدفق باستخدام الأحداث
- ✅ إنشاء منفذين مخصصين باستخدام @handler
- ✅ توجيه سير العمل بناءً على قرارات البشر
- ✅ بناء وكلاء ذكاء اصطناعي تفاعليين يتعاونون مع البشر

**هذا نمط أساسي لبناء أنظمة ذكاء اصطناعي موثوقة وقابلة للتحكم!** 🚀



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالترجمة البشرية الاحترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
